In [1]:
%load_ext autotime

In [35]:
import json
import time
from datetime import datetime

from etltools import s3
from test_client import LambdaCalculatorTestClient

In [3]:
client = LambdaCalculatorTestClient()

# Test Cost Breakdown

In [4]:
uid = '1000101'
pids = ['2820028008119', '2820088001036']

client.calculate_breakdown(uid, pids)

u'[{"picwell_id": "2820028008119", "uid": "1000101", "covered_breakdown": {"composite": 2730.6020000000003, "in_network": 2730.6020000000003, "categories": {"25": 350.0, "13": 106.19, "12": 721.0799999999999, "15": 179.73000000000002, "17": 680.0, "49": 12.794, "19": 80.0, "32": 0.0, "16": 0.0, "41": 0.0, "0": 0.0, "7": 520.8080000000001, "30": 80.0}, "out_network": 0.0}, "uncovered": 0.0, "deductible_breakdown": {"composite": 0.0, "in_network": 0.0, "categories": {"25": 0.0, "13": 0.0, "12": 0.0, "15": 0.0, "17": 0.0, "49": 0.0, "19": 0.0, "32": 0.0, "16": 0.0, "41": 0.0, "0": 0.0, "7": 0.0, "30": 0.0}, "out_network": 0.0}, "oop": 2730.6020000000003, "allowed": 14459.270000000004, "uncovered_breakdown": {"composite": 0.0, "in_network": 0.0, "categories": {"25": 0.0, "13": 0.0, "12": 0.0, "15": 0.0, "17": 0.0, "49": 0.0, "19": 0.0, "32": 0.0, "16": 0.0, "41": 0.0, "0": 0.0, "7": 0.0, "30": 0.0}, "out_network": 0.0}}, {"picwell_id": "2820088001036", "uid": "1000101", "covered_breakdown"

time: 595 ms


# Test Batch Calculation

In [4]:
uids = s3.read_json('s3n://picwell.sandbox.medicare/samples/philadelphia-2015')

print '{} uids read'.format(len(uids))

1352473 uids read


In [5]:
uids[:10]

[u'31970436701',
 u'1674435601',
 u'204451602',
 u'3158762201',
 u'595989402',
 u'894536502',
 u'30498125901',
 u'3173810801',
 u'1789308302',
 u'986282601']

In [32]:
requests_per_second = 100

for uid in uids[:100]:
#     client.calculate_async(uid, months=['01'])
    start_time = datetime.now()
    client.calculate_async(uid)
    print (datetime.now() - start_time).total_seconds()
    
    #time.sleep(1.0/requests_per_second)  

0.084332
0.158705
0.088289
0.186873
0.087814
0.152434
0.087913
0.099402
0.096584
0.129873
0.079152
0.061407
0.096875
0.104571
0.110631
0.133801
0.10081
0.076163
0.093521
0.135543
0.082093
0.126071
0.081005
0.086527
0.085486
0.087119
0.087784
0.093036
0.150434
0.140938
0.091262
0.110004
0.084024
0.071304
0.102804
0.209344
0.114169
0.148513
0.260177
0.100243
0.084655
0.086553
0.146651
0.119328
0.078969
0.084882
0.095827
0.259608
0.082847
0.088391
0.098038
0.088544
0.095789
0.096906
0.078592
0.090654
0.086977
0.084791
0.090596
0.096579
0.088741
0.089045
0.102221
0.115374
0.121692
0.087564
0.086235
0.090359
0.281282
0.128876
0.086773
0.131909
0.087728
0.078539
0.068069
0.09267
0.086539
0.153036
0.081993
0.076797
0.123167
0.182001
0.092552
0.094504
0.088238
0.09401
0.09419
0.090655
0.0851
0.087434
0.129234
0.088283
0.078164
0.091311
0.088593
0.085626
0.162438
0.11868
0.19269
0.167111


In [8]:
import schedule
%config IPCompleter.greedy=True

fewer_uids = uids[:1000]

1352473
1000
after pop
1352473
999


In [10]:
fewer_uids = uids[:1000]

def call_lambda_with_next_id():
    if len(fewer_uids) > 0:
        client.calculate_async(fewer_uids.pop())

schedule.every(.01).seconds.do(call_lambda_with_next_id)
#schedule.every().hour.do(job)
#schedule.every().day.at("10:30").do(job)
#schedule.every(5).to(10).minutes.do(job)
#schedule.every().monday.do(job)
#schedule.every().wednesday.at("13:15").do(job)

#while True:
#    schedule.run_pending()
#    time.sleep(1)
print('Done')

Done


In [12]:
import sched

In [26]:
scheduler = sched.scheduler(time.time, time.sleep)

def print_event(name):
    print 'EVENT:', time.time(), name

print 'START:', time.time()

start_time = time.time()

increment = .01

print 'plus 100 ms:', time.time() + .01

scheduler.enter(1, 1, print_event, ('first',))
scheduler.enter(5, 1, print_event, ('second',))

count = 1
start_time_for_task = increment

for uid in uids[:1000]:
    start_time_for_task = start_time_for_task + increment
    scheduler.enter(start_time_for_task, 1, print_event, ('call {}'.format(count),))
    count = count + 1


scheduler.run()

START: 1513179368.6
plus 100 ms: 1513179368.61
EVENT: 1513179368.63 call 1
EVENT: 1513179368.64 call 2
EVENT: 1513179368.65 call 3
EVENT: 1513179368.66 call 4
EVENT: 1513179368.67 call 5
EVENT: 1513179368.68 call 6
EVENT: 1513179368.69 call 7
EVENT: 1513179368.7 call 8
EVENT: 1513179368.71 call 9
EVENT: 1513179368.72 call 10
EVENT: 1513179368.73 call 11
EVENT: 1513179368.74 call 12
EVENT: 1513179368.75 call 13
EVENT: 1513179368.76 call 14
EVENT: 1513179368.77 call 15
EVENT: 1513179368.78 call 16
EVENT: 1513179368.79 call 17
EVENT: 1513179368.8 call 18
EVENT: 1513179368.81 call 19
EVENT: 1513179368.82 call 20
EVENT: 1513179368.83 call 21
EVENT: 1513179368.84 call 22
EVENT: 1513179368.85 call 23
EVENT: 1513179368.86 call 24
EVENT: 1513179368.87 call 25
EVENT: 1513179368.88 call 26
EVENT: 1513179368.89 call 27
EVENT: 1513179368.9 call 28
EVENT: 1513179368.91 call 29
EVENT: 1513179368.92 call 30
EVENT: 1513179368.93 call 31
EVENT: 1513179368.94 call 32
EVENT: 1513179368.95 call 33
EVENT: 1

EVENT: 1513179371.52 call 290
EVENT: 1513179371.53 call 291
EVENT: 1513179371.54 call 292
EVENT: 1513179371.55 call 293
EVENT: 1513179371.56 call 294
EVENT: 1513179371.57 call 295
EVENT: 1513179371.58 call 296
EVENT: 1513179371.59 call 297
EVENT: 1513179371.6 call 298
EVENT: 1513179371.61 call 299
EVENT: 1513179371.62 call 300
EVENT: 1513179371.63 call 301
EVENT: 1513179371.64 call 302
EVENT: 1513179371.65 call 303
EVENT: 1513179371.66 call 304
EVENT: 1513179371.67 call 305
EVENT: 1513179371.68 call 306
EVENT: 1513179371.69 call 307
EVENT: 1513179371.7 call 308
EVENT: 1513179371.71 call 309
EVENT: 1513179371.72 call 310
EVENT: 1513179371.73 call 311
EVENT: 1513179371.74 call 312
EVENT: 1513179371.75 call 313
EVENT: 1513179371.76 call 314
EVENT: 1513179371.77 call 315
EVENT: 1513179371.78 call 316
EVENT: 1513179371.79 call 317
EVENT: 1513179371.8 call 318
EVENT: 1513179371.81 call 319
EVENT: 1513179371.82 call 320
EVENT: 1513179371.83 call 321
EVENT: 1513179371.84 call 322
EVENT: 151317

EVENT: 1513179374.41 call 577
EVENT: 1513179374.42 call 578
EVENT: 1513179374.43 call 579
EVENT: 1513179374.44 call 580
EVENT: 1513179374.45 call 581
EVENT: 1513179374.46 call 582
EVENT: 1513179374.47 call 583
EVENT: 1513179374.48 call 584
EVENT: 1513179374.49 call 585
EVENT: 1513179374.5 call 586
EVENT: 1513179374.51 call 587
EVENT: 1513179374.52 call 588
EVENT: 1513179374.53 call 589
EVENT: 1513179374.54 call 590
EVENT: 1513179374.55 call 591
EVENT: 1513179374.56 call 592
EVENT: 1513179374.57 call 593
EVENT: 1513179374.58 call 594
EVENT: 1513179374.59 call 595
EVENT: 1513179374.6 call 596
EVENT: 1513179374.61 call 597
EVENT: 1513179374.62 call 598
EVENT: 1513179374.63 call 599
EVENT: 1513179374.64 call 600
EVENT: 1513179374.65 call 601
EVENT: 1513179374.66 call 602
EVENT: 1513179374.67 call 603
EVENT: 1513179374.68 call 604
EVENT: 1513179374.69 call 605
EVENT: 1513179374.7 call 606
EVENT: 1513179374.71 call 607
EVENT: 1513179374.72 call 608
EVENT: 1513179374.73 call 609
EVENT: 151317

EVENT: 1513179377.32 call 868
EVENT: 1513179377.33 call 869
EVENT: 1513179377.34 call 870
EVENT: 1513179377.35 call 871
EVENT: 1513179377.36 call 872
EVENT: 1513179377.37 call 873
EVENT: 1513179377.38 call 874
EVENT: 1513179377.39 call 875
EVENT: 1513179377.4 call 876
EVENT: 1513179377.41 call 877
EVENT: 1513179377.42 call 878
EVENT: 1513179377.43 call 879
EVENT: 1513179377.44 call 880
EVENT: 1513179377.45 call 881
EVENT: 1513179377.46 call 882
EVENT: 1513179377.47 call 883
EVENT: 1513179377.48 call 884
EVENT: 1513179377.49 call 885
EVENT: 1513179377.5 call 886
EVENT: 1513179377.51 call 887
EVENT: 1513179377.52 call 888
EVENT: 1513179377.53 call 889
EVENT: 1513179377.54 call 890
EVENT: 1513179377.55 call 891
EVENT: 1513179377.56 call 892
EVENT: 1513179377.57 call 893
EVENT: 1513179377.58 call 894
EVENT: 1513179377.59 call 895
EVENT: 1513179377.6 call 896
EVENT: 1513179377.61 call 897
EVENT: 1513179377.62 call 898
EVENT: 1513179377.63 call 899
EVENT: 1513179377.64 call 900
EVENT: 151317

# THREADING EXAMPLE

In [43]:
import threading

def call_lambda_with_id(uid, thread_num):
    print 'Thread {} started'.format(thread_num)
    client.calculate_async(uid)
    print 'Thread {} finished'.format(thread_num)
    return

#def worker(num):
#    """thread worker function"""
#    print 'Worker: %s' % num
#    return

def seconds_since(start_time):
    return (datetime.now() - start_time).total_seconds()

threads = []
start_threads = datetime.now()
thread_num = 1
for uid in uids[:100]:
    print '{}: Start thread {} for uid {}'.format(seconds_since(start_threads), thread_num, uid)
    t = threading.Thread(target=call_lambda_with_id, args=(uid,thread_num,))
    threads.append(t)
    t.start()
    time.sleep(1.0/requests_per_second)
    thread_num = thread_num + 1

print 'All threads finished in {}'.format(seconds_since(start_threads)) 

0.000174: Start thread 1 for uid 31970436701
Thread 1 started
0.013706: Start thread 2 for uid 1674435601
Thread 2 started
0.025329: Start thread 3 for uid 204451602
Thread 3 started
0.037906: Start thread 4 for uid 3158762201
Thread 4 started
0.052795: Start thread 5 for uid 595989402
Thread 5 started
0.064377: Start thread 6 for uid 894536502
Thread 6 started
0.077239: Start thread 7 for uid 30498125901
Thread 7 started
0.089229: Start thread 8 for uid 3173810801
Thread 8 started
0.10222: Start thread 9 for uid 1789308302
Thread 9 started
0.115155: Start thread 10 for uid 986282601
Thread 10 started
0.129907: Start thread 11 for uid 1784327003
Thread 11 started
0.141415: Start thread 12 for uid 246540301
Thread 12 started
0.153321: Start thread 13 for uid 3154052701
Thread 13 started
0.164647: Start thread 14 for uid 31917048601
Thread 14 started
0.176467: Start thread 15 for uid 617921001
Thread 15 started
0.187865: Start thread 16 for uid 3170818001
Thread 16 started
0.199487: Star